In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix
from sklearn import metrics


In [2]:
df = pd.read_csv("Dataset.csv", encoding = 'latin1')

In [3]:
new_df = df.drop(['OBJECTID1','File_tag','Provider','Date','Address_of_incident_4','CODE','NAME','ID_of__nearest__asset_5',
             'ID_of_nearest__polyphase__line_','Type_of_primary_asset__involved','Kind_of_fire_start_as_per_Claus',
             'Location__area_12','Fire__danger__rating_13','DNSP_record_number_14','OSIRIS___ESV__reference__number',
             'Fault__description_16','Overhead_conductors','BUFF_DIST','FOR_CODE','STATION_ID','CODE','NAME','WeatherStation',
             'Voltage_of_line_in_which__fire_','Network_categorisation_8','Phase_s__of__line_or__transform','FOR_TYPE','FOR_CAT'], axis = 1)

In [4]:
real_data_df = new_df.dropna()


In [5]:
real_data_df.head(3)

,DateOnly,Date_1,Time_2,Latitude_3,Longitude_3,Location_multiplier,Danger_multiplier,Product_of_multipliers,COVER,HEIGHT,FOREST,Elevation,Monthly Avg Rainfall,Monthly Mean Max Temperature,Monthly Mean Solar Exposure
1,24/01/2020,43854.00000,0.667361,-38.754127,143.667405,0.2,0.5,0.1,6,6,0,7.690627,129.4,26.9,21.8
3,20/02/2018,43151.69375,43151.693750,-38.669141,145.615890,0.2,1.0,0.2,6,6,0,25.977764,1.2,27,20.8
4,23/02/2018,43154.91458,43154.914580,-38.703816,145.819190,4.6,0.5,2.3,6,6,0,9.772949,1.2,27,20.8


In [7]:

real_data_df.shape

(1625, 15)

In [10]:

#Randomly select 50 records from the dataset

random.seed(10)
total_records = len(real_data_df.index)
imag_data_df = real_data_df.sample(n=total_records,replace=True)
imag_data_df.shape


(1625, 15)

In [11]:
imag_data_df.head(2)

,DateOnly,Date_1,Time_2,Latitude_3,Longitude_3,Location_multiplier,Danger_multiplier,Product_of_multipliers,COVER,HEIGHT,FOREST,Elevation,Monthly Avg Rainfall,Monthly Mean Max Temperature,Monthly Mean Solar Exposure
1515,30/01/2019,43495.66111,43495.66111,-36.334036,145.963279,4.6,0.5,2.3,6,6,0,169.551605,12.6,28.4,25.6
65,15/02/2020,43876.57292,43876.57292,-38.464289,146.088607,1.0,0.2,0.2,6,6,0,214.285584,75.8,24.4,17.9


In [182]:
# # # F-factor weight allocation: ground-fire (1.0) or other (0.2)

# # f.ground.w=1.0
# # f.other.w=1.0

# # G-factor weight allocation: LBRA (0.25), HBRA (1.0), REFCL (1.5), PRF (4.5)  
# Location Multiplier

# g.lbra.w=0.2
# g.hbra.w=1.0
# g.refcl.w=4.6,
# g.prf.w=19.8

# # T-factor weight allocation: No forecast (0.1), Low-Moderate (0.2), high (0.5), very high (1.0), severe (2.0), extreme (3.0), code red (5.0)

# Danger Multiplier

# t.noforecast.w=0.1
# t.low2mod.w=0.2
# t.high.w=0.5
# t.veryhigh.w=1.0
# t.severe.w=2.0
# t.extreme.w=3.5
# t.codered.w=5


# Create a Factor/case Matrix  we have 4 G-Factor and 7 T-factor so a total of 28 cases.

import itertools
g_factors = [0.2,1.0,4.6,19.8]
t_factors = [0.1,0.2,0.5,1,2.0,3.5,5]
cases = [g_factors,t_factors]
cases_list = list(itertools.product(*cases))
len(cases_list)
#cases_list

28

In [19]:
# data frame for cases real 

df_cases_real = pd.DataFrame(cases_list, columns=['g_factors', 't_factors'])
df_cases_real.insert((len(df_cases_real.columns)-1)+1 , 'case', range(1, 1 + len(df_cases_real)))
df_cases_real.insert((len(df_cases_real.columns)-1)+1 , 'count', 0)
df_cases_real.head(5)

,g_factors,t_factors,case,count
0,0.2,0.1,1,0
1,0.2,0.2,2,0
2,0.2,0.5,3,0
3,0.2,1.0,4,0
4,0.2,2.0,5,0


In [20]:
# data frame for cases imag 

df_cases_imag = pd.DataFrame(cases_list, columns=['g_factors', 't_factors'])
df_cases_imag.insert((len(df_cases_imag.columns)-1)+1 , 'case', range(1, 1 + len(df_cases_imag)))
df_cases_imag.insert((len(df_cases_imag.columns)-1)+1 , 'count', 0)
df_cases_imag.head(5)

,g_factors,t_factors,case,count
0,0.2,0.1,1,0
1,0.2,0.2,2,0
2,0.2,0.5,3,0
3,0.2,1.0,4,0
4,0.2,2.0,5,0


In [24]:
# classify records according to case type Real 

group_rows_real = pd.DataFrame(real_data_df.groupby(['Location_multiplier','Danger_multiplier']).size().reset_index(name='count'))
group_rows_real.head(5)

,Location_multiplier,Danger_multiplier,count
0,0.2,0.1,238
1,0.2,0.2,234
2,0.2,0.5,262
3,0.2,1.0,122
4,0.2,2.0,39


In [26]:
# classify records according to case type Imag

group_rows_imag = pd.DataFrame(imag_data_df.groupby(['Location_multiplier','Danger_multiplier']).size().reset_index(name='count'))
group_rows_imag.head(5)


,Location_multiplier,Danger_multiplier,count
0,0.2,0.1,251
1,0.2,0.2,245
2,0.2,0.5,246
3,0.2,1.0,123
4,0.2,2.0,35


In [96]:
#Calculate impact factor for Real Data frame  ### Imapact factor  (Cases*NumberofRecords) => C*N 

df_cases_real["IRU_real"] = df_cases_real["g_factors"] * df_cases_real["t_factors"] * df_cases_real["count"]  

In [97]:
#Calculate impact factor for Imag Data frame  ### Imapact factor  (Cases*NumberofRecords) => C*N

df_cases_imag["IRU_imag"] = df_cases_imag["g_factors"] * df_cases_imag["t_factors"] * df_cases_imag["count"] 

In [149]:
# adding grouped rows to Real cases data frame
for index_cases, row_cases in df_cases_real.iterrows():
    for index_records, row_records in group_rows_real.iterrows():
        if (row_cases['g_factors'] == row_records['Location_multiplier'] and  row_cases['t_factors'] == row_records['Danger_multiplier']):
            df_cases_real.loc[index_cases, 'count'] =   row_records['count'] 

df_cases_real

,g_factors,t_factors,case,count,IRU_real
0,0.2,0.1,1,238.0,4.76
1,0.2,0.2,2,234.0,9.36
2,0.2,0.5,3,262.0,26.20
3,0.2,1.0,4,122.0,24.40
4,0.2,2.0,5,39.0,15.60
5,0.2,3.5,6,16.0,11.20
6,0.2,5.0,7,0.0,0.00
7,1.0,0.1,8,78.0,7.80
8,1.0,0.2,9,86.0,17.20
9,1.0,0.5,10,106.0,53.00


In [148]:
# adding grouped rows to Imag cases data frame
for index_cases, row_cases in df_cases_imag.iterrows():
    for index_records, row_records in group_rows_imag.iterrows():
        if (row_cases['g_factors'] == row_records['Location_multiplier'] and  row_cases['t_factors'] == row_records['Danger_multiplier']):
            df_cases_imag.loc[index_cases, 'count'] =   row_records['count'] 

df_cases_imag


,g_factors,t_factors,case,count,IRU_imag
0,0.2,0.1,1,251.0,5.02
1,0.2,0.2,2,245.0,9.80
2,0.2,0.5,3,246.0,24.60
3,0.2,1.0,4,123.0,24.60
4,0.2,2.0,5,35.0,14.00
5,0.2,3.5,6,18.0,12.60
6,0.2,5.0,7,0.0,0.00
7,1.0,0.1,8,67.0,6.70
8,1.0,0.2,9,83.0,16.60
9,1.0,0.5,10,115.0,57.50


In [32]:
# get highest cases counts

# df_max_cases = df_cases.nlargest(3,columns=['count'])
# df_max_cases = df_max_cases.sort_values('case')
# df_max_cases

In [150]:
# Define where Cases Real and Cases Imag 
 
c_real = np.zeros(shape=[28,]) # cases Real
c_imag = np.zeros(shape=[28,]) # cases Imag
c0 = np.zeros(shape=[28,1])
c_real = df_cases_real[df_cases_real.columns[4]].to_numpy()
c_imag = df_cases_imag[df_cases_imag.columns[4]].to_numpy()

c0 = np.subtract(c_real,c_imag)
c0


array([ -0.26,  -0.44,   1.6 ,  -0.2 ,   1.6 ,  -1.4 ,   0.  ,   1.1 ,
         0.6 ,  -4.5 ,  -5.  ,   4.  ,   0.  ,   0.  ,  -5.52,  13.8 ,
       -13.8 ,  -9.2 ,   0.  ,  16.1 ,  23.  ,   1.98,   7.92,  29.7 ,
        19.8 ,   0.  ,  69.3 ,   0.  ])

In [422]:
 # Optimization Problem
 # Objective

import numpy as np
from scipy.optimize import minimize

#
x_penalty = 25000
# 1 M - [IRU(real) - IRU(img)] = 1M - (c_i * n_i (real) - c_i * n_i (img) ) * 25k 
def objective(x , sign=-1.0):
    return   (c0[0]+c0[1]+c0[2]+c0[3]+c0[4]+c0[5]+c0[6]+c0[7]+c0[8]+c0[9]+c0[10]+c0[11]+c0[12]+c0[13]+c0[14]+c0[15]+c0[16]+c0[17]+c0[18]+c0[19]+c0[20]+c0[21]+c0[22]+c0[23]+c0[24]+c0[25]+c0[26]+c0[27])


# sign*( 1000000 - (c0[0]+c0[1]+c0[2]+c0[3]+c0[4]+c0[5]+c0[6]+c0[7]+c0[8]+c0[9]+c0[10]+c0[11]+c0[12]+c0[13]+c0[14]+c0[15]+c0[16]+c0[17]+c0[18]+c0[19]+c0[20]+c0[21]+c0[22]+c0[23]+c0[24]+c0[25]+c0[26]+c0[27]) * x_penalty)


In [423]:
# Contraints 

def constraint1(x):
    return c0[1] - c0[0]     #C2- C1   c1 < c2

def constraint2(x):
    return c0[2] - c0[1]     #C3- C2   c2 < c3

def constraint3(x):
    return c0[3] - c0[2]     #C2- C1   c3 < c2

def constraint4(x):
    return c0[4] - c0[3]     #C3- C2   c4 < c3

def constraint5(x):
    return c0[5] - c0[4]     #C2- C1   c5 < c2

def constraint6(x):
    return c0[6] - c0[5]     #C3- C2   c6 < c3

def constraint7(x):
    return c0[7] - c0[6]     #C2- C1   c7 < c2

def constraint8(x):
    return c0[8] - c0[7]     #C3- C2   c8 < c3

def constraint9(x):
    return c0[9] - c0[8]     #C2- C1   c9 < c2

def constraint10(x):
    return c0[10] - c0[9]     #C3- C2   c2 < c3

def constraint11(x):
    return c0[11] - c0[10]     #C2- C1   c1 < c2

def constraint12(x):
    return c0[12] - c0[11]     #C3- C2   c2 < c3

def constraint13(x):
    return c0[13] - c0[12]     #C2- C1   c1 < c2

def constraint14(x):
    return c0[14] - c0[13]     #C3- C2   c2 < c3

def constraint15(x):
    return c0[15] - c0[14]     #C2- C1   c1 < c2

def constraint16(x):
    return c0[16] - c0[15]     #C3- C2   c2 < c3

def constraint17(x):
    return c0[17] - c0[16]     #C2- C1   c1 < c2

def constraint18(x):
    return c0[18] - c0[17]     #C3- C2   c2 < c3

def constraint19(x):
    return c0[19] - c0[18]     #C2- C1   c1 < c2

def constraint20(x):
    return c0[20] - c0[19]     #C3- C2   c2 < c3

def constraint21(x):
    return c0[21] - c0[20]     #C2- C1   c1 < c2

def constraint22(x):
    return c0[22] - c0[21]     #C3- C2   c2 < c3

def constraint23(x):
    return c0[23] - c0[22]     #C2- C1   c1 < c2

def constraint24(x):
    return c0[24] - c0[23]     #C3- C2   c2 < c3

def constraint25(x):
    return c0[25] - c0[24]     #C3- C2   c2 < c3

def constraint26(x):
    return c0[26] - c0[25]     #C2- C1   c1 < c2

def constraint27(x):
    return c0[27] - c0[26]     #C3- C2   c2 < c3 

def constraint28(x):
    return (c0[0]+c0[1]+c0[2]+c0[3]+c0[4]+c0[5]+c0[6]+c0[7]+c0[8]+c0[9]+c0[10]+c0[11]+c0[12]+c0[13]+c0[14]+c0[15]+c0[16]+c0[17]+c0[18]+c0[19]+c0[20]+c0[21]+c0[22]+c0[23]+c0[24]+c0[25]+c0[26]+c0[27]) * x_penalty - 1000000   #C3- C2   c2 < c3


con1 = {'type': 'ineq', 'fun': constraint1}
con2 = {'type': 'ineq', 'fun': constraint2}
con3 = {'type': 'ineq', 'fun': constraint3}
con4 = {'type': 'ineq', 'fun': constraint4}
con5 = {'type': 'ineq', 'fun': constraint5}
con6 = {'type': 'ineq', 'fun': constraint6}
con7 = {'type': 'ineq', 'fun': constraint7}
con8 = {'type': 'ineq', 'fun': constraint8}
con9 = {'type': 'ineq', 'fun': constraint9}
con10 = {'type': 'ineq', 'fun': constraint10}
con11 = {'type': 'ineq', 'fun': constraint11}
con12 = {'type': 'ineq', 'fun': constraint12}
con13 = {'type': 'ineq', 'fun': constraint13}
con14 = {'type': 'ineq', 'fun': constraint14}
con15 = {'type': 'ineq', 'fun': constraint15}
con16 = {'type': 'ineq', 'fun': constraint16}
con17 = {'type': 'ineq', 'fun': constraint17}
con18 = {'type': 'ineq', 'fun': constraint18}
con19 = {'type': 'ineq', 'fun': constraint19}
con20 = {'type': 'ineq', 'fun': constraint20}
con21 = {'type': 'ineq', 'fun': constraint21}
con22 = {'type': 'ineq', 'fun': constraint22}
con23 = {'type': 'ineq', 'fun': constraint23}
con24 = {'type': 'ineq', 'fun': constraint24}
con25 = {'type': 'ineq', 'fun': constraint25}
con26 = {'type': 'ineq', 'fun': constraint26}
con27 = {'type': 'ineq', 'fun': constraint27}
con28 = {'type': 'ineq', 'fun': constraint28}



In [424]:
#optimize

cons = ([con1,con2,con3,con4,con5,con6,con7,con8,con9,con10,con11,con12,con13,con14,con15,con16,con17,con18,con19,con20,con21,con22,con23,con24,con25,con26,con27, con28])
solution = minimize(objective,c0,method='SLSQP',constraints=cons)

In [433]:

x = solution.x

# show final objective
print('Final Objective: ' + str(objective(x)))

# print solution
print('Solution')
print('c1 = ' + str(x[0]))
print('c2 = ' + str(x[1]))
print('c3 = ' + str(x[3]))
print('c4 = ' + str(x[4]))
print('c5 = ' + str(x[5]))
print('c6 = ' + str(x[6]))
print('c7 = ' + str(x[7]))
print('c8 = ' + str(x[8]))
print('c9 = ' + str(x[9]))
print('c10 = ' + str(x[10]))
print('c11 = ' + str(x[11]))
print('c12 = ' + str(x[12]))
print('c13 = ' + str(x[13]))
print('c14 = ' + str(x[14]))
print('c15 = ' + str(x[15]))
print('c16 = ' + str(x[16]))
print('c17 = ' + str(x[17]))
print('c18 = ' + str(x[18]))
print('c19 = ' + str(x[19]))
print('c20 = ' + str(x[20]))
print('c21 = ' + str(x[21]))
print('c22 = ' + str(x[22]))
print('c23 = ' + str(x[23]))
print('c24 = ' + str(x[24]))
print('c25 = ' + str(x[25]))
print('c26 = ' + str(x[26]))
print('c27 = ' + str(x[27]))


Final Objective: 150.18000000000006
Solution
c1 = -0.2600000000000007
c2 = -0.4400000000000013
c3 = -0.1999999999999993
c4 = 1.6000000000000014
c5 = -1.4000000000000004
c6 = 0.0
c7 = 1.1000000000000005
c8 = 0.5999999999999979
c9 = -4.5
c10 = -5.0
c11 = 4.0
c12 = 0.0
c13 = 0.0
c14 = -5.519999999999996
c15 = 13.799999999999997
c16 = -13.799999999999955
c17 = -9.199999999999989
c18 = 0.0
c19 = 16.099999999999998
c20 = 23.0
c21 = 1.9800000000000004
c22 = 7.919999999999998
c23 = 29.700000000000003
c24 = 19.799999999999997
c25 = 0.0
c26 = 69.3
c27 = 0.0


In [434]:
# dataframe with values of Cases

df_cases_values = pd.DataFrame(x, columns=['values'])
df_cases_values.insert((len(df_cases_values.columns)-1)+1 , 'case', range(1, 1 + len(df_cases_values)))
df_cases_values

,values,case
0,-0.26,1
1,-0.44,2
2,1.60,3
3,-0.20,4
4,1.60,5
5,-1.40,6
6,0.00,7
7,1.10,8
8,0.60,9
9,-4.50,10


In [440]:
test_real_df = pd.DataFrame(columns=['count', 'c', 'IRU'])
test_real_df['count'] = df_cases_real['count']
test_real_df['c'] = df_cases_values['values']
test_real_df['IRU'] = test_real_df['count'] * test_real_df['c']
sum_iru_real = np.sum(test_real_df['IRU'])


test_imag_df = pd.DataFrame(columns=['count', 'c', 'IRU'])
test_imag_df['count'] = df_cases_imag['count']
test_imag_df['c'] = df_cases_values['values']
test_imag_df['IRU'] = test_imag_df['count'] * test_imag_df['c']
sum_iru_imag = np.sum(test_imag_df['IRU'])

np.abs(sum_iru_real - sum_iru_imag) * 25000

18504499.999999978

In [441]:
sum_iru_real

-731.1799999999946

In [442]:
sum_iru_imag

-1471.3599999999938

In [444]:
np.abs(sum_iru_real - sum_iru_imag) * 25000

18504499.999999978

In [439]:
test_real_df.head(20)

,count,c,IRU
0,238.0,-0.26,-61.88
1,234.0,-0.44,-102.96
2,262.0,1.60,419.20
3,122.0,-0.20,-24.40
4,39.0,1.60,62.40
5,16.0,-1.40,-22.40
6,0.0,0.00,0.00
7,78.0,1.10,85.80
8,86.0,0.60,51.60
9,106.0,-4.50,-477.00


In [428]:
# Location Multiplier

# g.lbra.w=0.2
# g.hbra.w=1.0
# g.refcl.w=4.6,
# g.prf.w=19.8

# # T-factor weight allocation: No forecast (0.1), Low-Moderate (0.2), high (0.5), very high (1.0), severe (2.0), extreme (3.0), code red (5.0)

# Danger Multiplier

# t.noforecast.w=0.1
# t.low2mod.w=0.2
# t.high.w=0.5
# t.veryhigh.w=1.0
# t.severe.w=2.0
# t.extreme.w=3.5
# t.codered.w=5

import itertools
g_factors_names = ['g_lbra','g_hbra','g_refcl','g_prf']
t_factors_names = ['t_noforecast','t_low2mod','t_high','t_veryhigh','t_severe','t_extreme','t_codered']
cases_names = [g_factors_names,t_factors_names]
cases_list_names = list(itertools.product(*cases_names))
len(cases_list_names)
cases_list_names[0][0]

'g_lbra'

In [429]:
zeroes = np.zeros(shape=[28,11]) 
df_case_matrix = pd.DataFrame(zeroes,columns=['g_lbra','g_hbra','g_refcl','g_prf','t_noforecast','t_low2mod','t_high','t_veryhigh','t_severe','t_extreme','t_codered'])

df_case_matrix

,g_lbra,g_hbra,g_refcl,g_prf,t_noforecast,t_low2mod,t_high,t_veryhigh,t_severe,t_extreme,t_codered
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [343]:

i = 0
for g_fac,t_fac in cases_list_names:
    #print(g_fac,t_fac)
    df_case_matrix.loc[[i],[g_fac,t_fac]] = 1
    i = i+1
            
df_case_matrix

,g_lbra,g_hbra,g_refcl,g_prf,t_noforecast,t_low2mod,t_high,t_veryhigh,t_severe,t_extreme,t_codered
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [351]:
df_case_matrix['case']= df_cases_values['case']
df_case_matrix['values']= df_cases_values['values']
df_case_matrix

,g_lbra,g_hbra,g_refcl,g_prf,t_noforecast,t_low2mod,t_high,t_veryhigh,t_severe,t_extreme,t_codered,cases,values
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,-0.26
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2,-0.44
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,1.60
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4,-0.20
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,1.60
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6,-1.40
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7,0.00
7,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8,1.10
8,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9,0.60
9,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10,-4.50


In [353]:
df_matrix_max = df_case_matrix.nlargest(11,'values')
df_matrix_max

,g_lbra,g_hbra,g_refcl,g_prf,t_noforecast,t_low2mod,t_high,t_veryhigh,t_severe,t_extreme,t_codered,cases,values
26,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,27,69.30
23,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,24,29.70
20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21,23.00
24,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,25,19.80
19,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20,16.10
15,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16,13.80
22,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,23,7.92
11,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,12,4.00
21,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,22,1.98
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,1.60


In [382]:
numpy_array = df_matrix_max.to_numpy()
A, b = numpy_array[:,0:11], numpy_array[:, 12]

A.shape


(11, 11)

In [384]:
z= np.linalg.lstsq(A,b)


C:\Users\Waqar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [392]:
z

(array([-2.41636364e+00, -1.63636364e-02,  2.02363636e+00,  2.56836364e+01,
        -2.37036364e+01, -2.99363636e+00,  4.01636364e+00, -5.88363636e+00,
         4.01636364e+00,  2.88463636e+01,  2.09763636e+01]),
 array([], dtype=float64),
 10,
 array([2.49054160e+00, 2.00694676e+00, 1.86195028e+00, 1.50719878e+00,
        1.41421356e+00, 1.16246135e+00, 1.00000000e+00, 1.00000000e+00,
        7.21532991e-01, 3.98665229e-01, 8.42764748e-17]))